# Scripts para crear los archivos txt con los inserts

Importanción de librerías:

In [193]:
import pandas as pd
import numpy as np
from itertools import chain
from database import Database

db = Database('../db/fifa25.db')

Ruta global y lectura de los datasets

In [194]:
BASE_PATH = './../docs/'
df_famales = pd.read_csv(f'{BASE_PATH}female_players.csv', sep=',')
df_males = pd.read_csv(f'{BASE_PATH}male_players.csv', sep=',')
df_all = pd.read_csv(f'{BASE_PATH}all_players.csv', sep=',')
df_all.columns

C:\Users\Yacky Ramírez\AppData\Local\Temp\ipykernel_21240\516013953.py:4: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  df_all = pd.read_csv(f'{BASE_PATH}all_players.csv', sep=',')


Index(['Unnamed: 0', 'Rank', 'Name', 'OVR', 'PAC', 'SHO', 'PAS', 'DRI', 'DEF',
       'PHY', 'Acceleration', 'Sprint Speed', 'Positioning', 'Finishing',
       'Shot Power', 'Long Shots', 'Volleys', 'Penalties', 'Vision',
       'Crossing', 'Free Kick Accuracy', 'Short Passing', 'Long Passing',
       'Curve', 'Dribbling', 'Agility', 'Balance', 'Reactions', 'Ball Control',
       'Composure', 'Interceptions', 'Heading Accuracy', 'Def Awareness',
       'Standing Tackle', 'Sliding Tackle', 'Jumping', 'Stamina', 'Strength',
       'Aggression', 'Position', 'Weak foot', 'Skill moves', 'Preferred foot',
       'Height', 'Weight', 'Alternative positions', 'Age', 'Nation', 'League',
       'Team', 'play style', 'url', 'GK Diving', 'GK Handling', 'GK Kicking',
       'GK Positioning', 'GK Reflexes'],
      dtype='object')

#### Llenar play_styles

Vamos a leer los <b>play style</b> y los vamos a colocar en una lista. Esta lista no deberá incluir repetidos. Generar un .txt con los inserts con el contenido de la lista creada para usarlos en esta tabla.

```sql
CREATE TABLE play_styles (
  id	INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
  name	VARCHAR(30)
);
```

In [195]:
rows = df_all['play style'].str.split(',')
play_styles = []
for array_str in rows:
    if isinstance(array_str, list):
        for style in array_str:
            # remove '* '
            if style[0] == ' ':
                style = style[1:]
            # remove ' *'
            if style[len(style) - 1] == ' ':
                style = style[:-1]
            if style not in play_styles:
                play_styles.append(style)
text = '-- migrate:up \n\n'
i = 1
for style in play_styles:
    text = text + f"INSERT INTO play_styles (id, name) VALUES ({i}, '{style}');\n"
    i = i + 1
text = text + '\n-- migrate:down \n\nDELETE FROM play_styles;'
with open(f'{BASE_PATH}play_styles_inserts.sql', 'w') as archivo:
    archivo.write(text)

#### Llenar nations

Vamos a leer los <b>natios</b> y los vamos a colocar en una lista. Esta lista no deberá incluir repetidos. Generar un .txt con los inserts con el contenido de la lista creada para usarlos en esta tabla.

```sql
CREATE TABLE nations (
  id	INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
  name	VARCHAR(40)
);
```

In [196]:
nations = sorted(list(df_all['Nation'].unique()))
text = '-- migrate:up \n\n'
i = 1
for name in nations:
    text = text + f"INSERT INTO nations (id, name) VALUES ({i}, '{name.replace('\'', '\'\'')}');\n"
    i = i + 1
text = text + '\n-- migrate:down \n\nDELETE FROM nations;'
with open(f'{BASE_PATH}nations_inserts.sql', 'w') as archivo:
    archivo.write(text)

#### Llenar positions

Vamos a leer los <b>positions</b> y los vamos a colocar en una lista. Esta lista no deberá incluir repetidos. Generar un .txt con los inserts con el contenido de la lista creada para usarlos en esta tabla.

```sql
CREATE TABLE positions (
  id	INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
  name	VARCHAR(40)
);
```

In [197]:
# 'Position'  'Alternative positions'

positions = sorted(list(df_all['Position'].unique()))
positions
alternative_positions = df_all['Alternative positions']

rows = df_all['Alternative positions'].str.split(',')
for array_str in rows:
    if isinstance(array_str, list):
        for position in array_str:
            # remove '* '
            if position[0] == ' ':
                position = position[1:]
            # remove ' *'
            if position[len(position) - 1] == ' ':
                position = position[:-1]
            if position not in positions:
                positions.append(position)

text = '-- migrate:up \n\n'
i = 1
for name in positions:
    text = text + f"INSERT INTO positions (id, name) VALUES ({i}, '{name.replace('\'', '\'\'')}');\n"
    i = i + 1
text = text + '\n-- migrate:down \n\nDELETE FROM positions;'
with open(f'{BASE_PATH}positions_inserts.sql', 'w') as archivo:
    archivo.write(text)
    

#### Llenar leagues

Vamos a leer los <b>leagues</b> y los vamos a colocar en una lista. Esta lista no deberá incluir repetidos. Generar un .txt con los inserts con el contenido de la lista creada para usarlos en esta tabla.

```sql
CREATE TABLE leagues (
  id	INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
  name	VARCHAR(30),
  nation_id	INTEGER,
  FOREIGN KEY (nation_id) REFERENCES nations (id)
);
```

In [206]:
df_leagues_nations = pd.read_csv(f'{BASE_PATH}league_nation.csv', sep=',')
leagues = sorted(list(df_all['League'].unique()))
text = '-- migrate:up \n\n'
i = 1
for index, row in df_leagues_nations.iterrows():
    rs = db.fetchone(f"SELECT id FROM nations WHERE name = '{row['Nation']}'")
    if rs is not None:
        nation_id = rs['id']
        name = row['League']
        text = text + f"INSERT INTO leagues (id, name, nation_id) VALUES ({i}, '{name.replace('\'', '\'\'')}', {nation_id});\n"
        i = i + 1
        #print(f"Index: {index}, League: {row['League']}, Nation: {row['Nation']}, NationID: {nation_id}")
    else:
        print(f"Nation '{row['Nation']}' not found in the database.")
text = text + '\n-- migrate:down \n\nDELETE FROM leagues;'
with open(f'{BASE_PATH}leagues_inserts.sql', 'w', encoding='utf-8') as archivo:
    archivo.write(text)